In [ ]:
import warnings

warnings.filterwarnings('ignore')
%reload_ext autoreload
%autoreload 2

import base64
import datetime
import io
import json
import sys
import textwrap
from pathlib import Path
from typing import List, Union

import basedosdados as bd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import google.generativeai as genai
from langchain import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI, chat_models
from pydantic import BaseModel, Field

from IPython.display import Markdown
from PIL import Image, ImageDraw, ImageFont

sys.path.insert(0, "../../")
from utils_sheets import save_data_in_sheets

bd.config.billing_project_id = 'rj-escritorio-dev'
bd.config.from_file = True
bd.__version__


pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200


from vision_ai_api import APIVisionAI

import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
# GOOGLE_API_KEY = ""
PROJECT_ID = "rj-escritorio-dev"
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

- storage images: https://console.cloud.google.com/storage/browser/datario-public/flooding_detection?project=datario
- imagens figma: https://www.figma.com/file/Qv89NLopXS60Lqf3XfTZiN/Untitled?type=design&node-id=3-44&mode=design&t=3a4g8D4QLiDQ8f3i-0
- langchain ref: https://python.langchain.com/docs/integrations/platforms/google


In [ ]:
def get_image_link_from_storage():
    dataset_id = "flooding_detection"
    table_id = "classified_images"
    st = bd.Storage(dataset_id=dataset_id, table_id=table_id)
    blobs = (
        st.client["storage_staging"]
        .bucket("datario-public")
        .list_blobs(prefix=f"{dataset_id}/{table_id}")
    )

    url_list = []
    for blob in blobs:
        url = str(blob.public_url)
        if "." in url.split("/")[-1]:
            url_list.append(url)
    return url_list


def get_urls_and_labels():
    urls = get_image_link_from_storage()
    labels = [
        {"path": "images_with_label/flood", "label": True, "object": "alagamento"},
        {"path": "images_with_label/no_flood", "label": False, "object": "alagamento"},
    ]
    data = []
    for url in urls:
        for item in labels:
            if item.get("path") in str(url):
                data.append(
                    {
                        "object": item.get("object"),
                        "label": item.get("label"),
                        "image_url": url,
                    }
                )
    return pd.DataFrame(data)


def balance_and_sample(df, N):
    # Get the minimum count of the two labels
    min_count = min(df["label"].value_counts())

    # Balance the DataFrame
    df_balanced = pd.concat(
        [df[df["label"] == True].head(min_count), df[df["label"] == False].head(min_count)]
    )
    df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
    # Sample N rows
    if N > len(df_balanced):
        print(
            f"Requested number of samples ({N}) is more than the available balanced dataset size ({len(df_balanced)})."
        )
        return df_balanced
    return df_balanced.head(N)


def get_image_from_url(image_url):
    response = requests.get(image_url)
    img = Image.open(io.BytesIO(response.content))
    img.thumbnail((640, 480))
    return img

In [ ]:
from langchain_core.messages import HumanMessage
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, chat_models

from langchain.output_parsers import PydanticOutputParser


class Object(BaseModel):
    object: str = Field(description="The object from the objects table")
    label_explanation: str = Field(
        description="Highly detailed visual description of the image given the object context"
    )
    label: Union[bool, str, None] = Field(
        description="Label indicating the condition or characteristic of the object"
    )


class ObjectFactory:
    @classmethod
    def generate_sample(cls) -> Object:
        return Object(
            object="<Object from objects table>",
            label_explanation="<Visual description of the image given the object context>",
            label="<Selected label from objects table>",
        )


class Output(BaseModel):
    objects: List[Object]


class OutputFactory:
    @classmethod
    def generate_sample(cls) -> Output:
        return Output(objects=[ObjectFactory.generate_sample()])


class OutputFactory:
    @classmethod
    def generate_sample(cls) -> Output:
        return Output(objects=[ObjectFactory.generate_sample()])


def get_parser():

    # Create the output parser using the Pydantic model
    output_parser = PydanticOutputParser(pydantic_object=Output)

    # Valid JSON string
    output_example_str = str(OutputFactory().generate_sample().dict()).replace("'", '"')

    output_example_str = textwrap.dedent(output_example_str)
    output_example = output_parser.parse(output_example_str)
    output_example_parsed = json.dumps(output_example.dict(), indent=4)

    output_schema = json.loads(output_parser.pydantic_object.schema_json())
    output_schema_parsed = json.dumps(output_schema, indent=4)

    return output_parser, output_schema_parsed, output_example_parsed


def get_objects_table(
    url: str = "https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=1672006844",
):
    request_url = url.replace("edit#gid=", "export?format=csv&gid=")
    response = requests.get(request_url)
    dataframe = pd.read_csv(io.StringIO(response.content.decode("utf-8")), dtype=str)
    dataframe["labels"] = dataframe["labels"].fillna("null")
    objects_table_md = dataframe.to_markdown(index=False)
    objects_labels = (
        dataframe[["object", "labels"]]
        .groupby(by=["object"], sort=False)["labels"]
        .apply(lambda x: ", ".join(x))
        .reset_index()
    )
    objects_labels["labels"] = objects_labels["labels"].str.replace("true, false", "bool")
    objects_labels_md = objects_labels.to_markdown(index=False)
    objects_labels_md = objects_labels_md
    return objects_table_md, objects_labels_md


def get_prompt():

    template = """
                **Role:**

                You are tasked with analyzing a CCTV image of Rio de Janeiro to identify potential city issues. Your analysis should focus on several key elements defined in an 'Objects Table', which includes criteria and identification guides for various objects and conditions that may impact city life. The goal is to classify these elements and provide a clear, concise description for each, enabling human CCTV operators to quickly understand and act upon the information. All objects from the 'Objects Table' MUST HAVE one entry in the output.

                **Objects Table**

                {objects_table_md}

                **Thought Process**

                    1. Use the criteria and identification_guide as context to describe the visual features of each object and fill the label_explanation output. Do not simply repeat the criteria or identification_guide but use then as context for your description.
                    2. Based on your visual description and analysis, select the most accurate label for each object from the options provided in the 'Objects Table'. You must use only the provided  labels!
                    3. Ensure that EVERY OBJECT from the objects table HAS ONE ENTRY in the output with the respective label and detailed description.
                    4. Return the output.   


                **Input:**
                A CCTV image.

                **Output:**

                    - Format the output as a JSON instance following the provided schema. 

                **Output Schema:**


                ```json
                {output_schema}
                ```


                **Example Output:**

                    - For each object analyzed, your output should resemble the following format:


                ```json
                {output_example}
                ```

                Now classify the image bellow:

    """

    template = vision_ai_api._get("/prompts").get("items")[0].get("prompt_text")

    prompt_template = PromptTemplate(
        input_variables=["objects_table_md", "output_schema", "output_example"], template=template
    )

    _, output_schema, output_example = get_parser()
    objects_table_md, _ = get_objects_table()

    filled_prompt = prompt_template.format(
        objects_table_md=objects_table_md,
        output_schema=output_schema,
        output_example=output_example,
    )
    filled_prompt = filled_prompt.replace("                    ", "")

    return filled_prompt, template


def gemini_pro_vision_langchain(
    image_url, prompt, max_output_token=300, temperature=0.4, top_k=32, top_p=1
):
    llm = ChatGoogleGenerativeAI(
        model="gemini-pro-vision",
        google_api_key=GOOGLE_API_KEY,
        max_output_token=max_output_token,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        stream=True,
    )

    content = [{"type": "text", "text": prompt}, {"type": "image_url", "image_url": image_url}]
    message = HumanMessage(content=content)
    return llm.invoke([message])


def gemini_pro_vision_google(
    image_url, prompt, max_output_token=300, temperature=0.4, top_k=32, top_p=1
):
    image_response = requests.get(image_url)
    image = Image.open(io.BytesIO(image_response.content))
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel("gemini-pro-vision")
    responses = model.generate_content(
        contents=[prompt, image],
        generation_config={
            "max_output_tokens": max_output_token,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
        },
        stream=True,
    )

    responses.resolve()
    ai_response = responses.text
    return ai_response


def gemini_pro_vision_vertex(
    image_url, prompt, max_output_token=300, temperature=0.4, top_k=32, top_p=1
):
    image_response = requests.get(image_url)
    image_type = image_url.split(".")[-1]
    model = GenerativeModel("gemini-pro-vision")
    responses = model.generate_content(
        contents=[prompt, Part.from_data(image_response.content, f"image/{image_type}")],
        generation_config={
            "max_output_tokens": max_output_token,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
        },
    )
    ai_response = responses.text
    return ai_response


def predict_and_save(
    save_data,
    output_parser,
    image_url,
    prompt,
    max_output_token,
    temperature,
    top_k,
    top_p,
    experiment_name,
    experiment_datetime,
    predictions_path,
):
    response = gemini_pro_vision_vertex(
        image_url=image_url,
        prompt=prompt,
        max_output_token=max_output_token,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )
    response_parsed = output_parser.parse(response)
    response_parsed = response_parsed.dict()

    save_data_in_sheets(
        save_data=save_data,
        data={
            "prompt": prompt,
            "max_output_token": max_output_token,
            "temperature": temperature,
            "top_k": top_k,
            "top_p": top_p,
            "experiment_name": experiment_name,
            "experiment_datetime": experiment_datetime,
            "true_object": "",
            "response": response_parsed,
            "image_url": image_url,
            "image": f'=IMAGE("{image_url}")',
        },
        data_url="https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=436224340",
        prompt_url="https://docs.google.com/spreadsheets/d/122uOaPr8YdW5PTzrxSPF-FD0tgco596HqgB7WK7cHFw/edit#gid=1779223884",
    )

    pd.DataFrame([{"image_url": image_url}]).to_csv(
        path_or_buf=predictions_path,
        index=False,
        header=not predictions_path.exists(),
        mode="a",
    )

    return response, response_parsed

In [ ]:
def get_urls_from_path(url_path):
    urls = get_image_link_from_storage()
    data = []
    for url in urls:
        if url_path in str(url):
            data.append(
                {
                    "image_url": url,
                }
            )
    return pd.DataFrame(data)


with open("secrets.json") as f:
    s = json.load(f)

vision_ai_api = APIVisionAI(
    username=s.get("username"),
    password=s.get("password"),
)


# df = get_urls_from_path(url_path="images_predicted_as_flood")
df = get_urls_from_path(url_path="/")
experiment_name = "test-object-label-table"
experiment_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
prompt, prompt_template = get_prompt()
max_output_token = 2048
temperature = 0.2
top_k = 32
top_p = 1
retry = 5

In [ ]:
predictions_path = Path(f"./data/predictions/{experiment_name}__{experiment_datetime}.csv")

if predictions_path.exists():
    predictions = pd.read_csv(predictions_path)
    predictions_list = predictions["image_url"].tolist()
else:
    predictions_list = []

predictions_list = []
output_parser, output_schema, output_example = get_parser()

for index, row in df.head(1).iterrows():
    image_url = row["image_url"]
    retry_count = 0
    while retry_count <= retry:
        try:
            if image_url not in predictions_list:
                response, response_parsed = predict_and_save(
                    save_data=True,
                    output_parser=output_parser,
                    image_url=image_url,
                    prompt=prompt,
                    max_output_token=max_output_token,
                    temperature=temperature,
                    top_k=top_k,
                    top_p=top_p,
                    experiment_name=experiment_name,
                    experiment_datetime=experiment_datetime,
                    predictions_path=predictions_path,
                )

                print(f"{index} - {len(df)}")
                # print(json.dumps(response_parsed, indent=4))
                # display(get_image_from_url(image_url))
            else:
                print(f"{index} - {len(df)}: already predicted")

            retry_count = retry + 1
        except Exception as e:
            retry_count += 1
            print(f"{index} - {len(df)}: Error.. Retrying:{retry_count}\n {e}\n\n\n\nAI Response:")
            print(response)

In [ ]:
model = genai.GenerativeModel("gemini-pro-vision")
model.generate_content()

In [ ]:
genai.configure()

In [ ]:
genai.configure()

In [ ]:
camera_id = "001477"
N = 100
for i, image_url in enumerate(
    N * [f"https://storage.googleapis.com/datario-public/vision-ai/staging/{camera_id}.png"]
):
    retry_count = 0
    while retry_count <= retry:
        try:
            response, response_parsed = predict_and_save(
                save_data=True,
                output_parser=output_parser,
                image_url=image_url,
                prompt=prompt,
                max_output_token=max_output_token,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
            )

            print(f"{i}")
            # print(json.dumps(response_parsed, indent=4))
            # display(get_image_from_url(image_url))
            retry_count = retry + 1
        except Exception as e:
            retry_count += 1
            print(f"{i}: Error.. Retrying:{retry_count}\n {e}\n\n\n\nAI Response:")
            print(response.content)

In [ ]:
len(response_parsed)

In [ ]:
output_parser.parse(response_parsed)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)
import seaborn as sns


true_values = df_final["flood"].tolist()
predicted_values = df_final["ai_label"].tolist()

# Calculate metrics
accuracy = accuracy_score(true_values, predicted_values)
precision = precision_score(true_values, predicted_values, pos_label=True)
recall = recall_score(true_values, predicted_values, pos_label=True)
f1 = f1_score(true_values, predicted_values, pos_label=True)
conf_matrix = confusion_matrix(true_values, predicted_values)

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


cm = confusion_matrix(true_values, predicted_values)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["False", "True"],
    yticklabels=["False", "True"],
)
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.title("Confusion Matrix")
plt.show()


# resize_factor = 3
# imgs = 10
# time = 1:00
# Accuracy: 0.7
# Precision: 0.6666666666666666
# Recall: 0.5
# F1 Score: 0.5714285714285715

# resize_factor = 1
# imgs = 10
# time = 1:15
# Accuracy: 0.8
# Precision: 0.75
# Recall: 0.75
# F1 Score: 0.75


# resize_factor = 5
# imgs = 10
# time = 1:20
# Accuracy: 0.8
# Precision: 0.75
# Recall: 0.75
# F1 Score: 0.75

In [ ]:
df_final["miss"] = np.where(df_final["flood"] == df_final["ai_label"], False, True)
mask = df_final["miss"] == True
miss = df_final[mask]
miss_imgs = miss["base64"].tolist()
miss_ai_labels = miss["ai_label"].tolist()


for base64_image, ai_label in zip(miss_imgs, miss_ai_labels):
    print(f"AI classyfy as: {ai_label}")
    display_img(base64_image)

You are a highly skilled prompt engineering focus in create prompts for CCTV image recognition. Your task is to optimize and refine a provided example prompt.

you output is a diff between the provided prompt and the new optmized prompt

shall we start?
